In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model.classification import *
from src import *
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

device = torch.device(f'cuda:1' if torch.cuda.is_available() else 'cpu')

In [3]:
files = [
    '20190819-Kutna Hora-L1-out-MVI_0007',
    '20190819-Kutna Hora-L3-in-MVI_0005',
    '20190819-Kutna Hora-L3-out-MVI_0008',
    '20190819-Kutna Hora-L4-in-MVI_0013',
    '20190819-Kutna Hora-L6-out-MVI_0017',
    '20190819-Kutna Hora-L7-out-MVI_0032',
    '20190819-Kutna Hora-L8-in-MVI_0045',
    '20190819-Kutna Hora-L9-in-MVI_0043',
    '20190819-Kutna Hora-L10-out-SDV_1888',
    '20190819-Kutna Hora-L13-in-MVI_0006',
    '20190819-Kutna Hora-L13-out-MVI_0018',
    '20190819-Kutna Hora-L14-out-MVI_0005',
    '20190819-Kutna Hora-L15-out-MVI_0012',
    '20190819-Kutna Hora-L18-in-MVI_0030',
    '20190819-Ricany-L2-in-MVI_0006',
    '20190819-Ricany-L2-out-MVI_0005',
    '20190819-Ricany-L3-in-MVI_0006',
    '20190819-Ricany-L6-in-MVI_0008',
    '20190819-Ricany-L6-out-MVI_0011',
    '20190819-Ricany-L7-in-MVI_0008',
    '20190819-Ricany-L7-out-MVI_0013',
    '20190819-Ricany-L8-in-MVI_0009',
    '20190819-Ricany-L8-out-MVI_0013',
    '20190819-Ricany-L9-in-MVI_0008',
    '20190819-Ricany-L9-out-MVI_0011'
]

In [4]:
datapool = DataPool(files, window_length=6, split_ratio=0.75, tqdm=tqdm)

video loading:   0%|          | 0/25 [00:00<?, ?it/s]

In [5]:
uuid = 1636031305
# suffix = 'diff'
# suffix = 'mae'
suffix = 'interval'
model, params = load_model(uuid, suffix)
transform = create_transformation(params)

In [6]:
def test(is_trn=True):
    outputs = []

    for video in tqdm(datapool):
        n_events = video.get_events_count(is_trn)
        from_time, till_time = video.get_from_till_time(is_trn)

        predictions = validate(video.signal, model, transform, params, from_time=from_time, till_time=till_time, classification=True)

        n_predicted = predictions.sum()
        rvce = np.abs(n_predicted - n_events) / n_events
        error = n_events - n_predicted
        outputs.append([f'{rvce:.3f}', error, n_events, f'{video.file} [{from_time:.0f}, {till_time:.0f}]'])

    outputs = np.array(outputs)
    idx = np.argsort(outputs[:, 0].astype(float))
    outputs = outputs[idx]

    rvce = outputs[:, 0].astype(float).mean()

    print('rvce', rvce)
    
    return outputs

def print_outputs(outputs):
    print(' rvce | error | n_events | file')
    print('– ' * 42)
    for row in outputs:
        row[0]
        print(f'{row[0]} | {row[1]:>5} | {row[2]:>8} | {row[3]}')

In [7]:
trn_outputs = test(is_trn=True)
val_outputs = test(is_trn=False)

  0%|          | 0/25 [00:00<?, ?it/s]

rvce 0.0038


  0%|          | 0/25 [00:00<?, ?it/s]

rvce 0.08276


In [8]:
print_outputs(trn_outputs)

 rvce | error | n_events | file
– – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – 
0.000 |     0 |      240 | 20190819-Kutna Hora-L1-out-MVI_0007 [0, 1554]
0.000 |     0 |      393 | 20190819-Ricany-L7-in-MVI_0008 [0, 1560]
0.000 |     0 |      605 | 20190819-Ricany-L6-out-MVI_0011 [0, 1566]
0.000 |     0 |      222 | 20190819-Kutna Hora-L14-out-MVI_0005 [0, 1554]
0.000 |     0 |       13 | 20190819-Kutna Hora-L13-out-MVI_0018 [0, 1680]
0.000 |     0 |        6 | 20190819-Kutna Hora-L13-in-MVI_0006 [0, 1620]
0.000 |     0 |      192 | 20190819-Kutna Hora-L10-out-SDV_1888 [0, 2820]
0.000 |     0 |        8 | 20190819-Kutna Hora-L15-out-MVI_0012 [0, 1548]
0.000 |     0 |      169 | 20190819-Kutna Hora-L8-in-MVI_0045 [0, 1446]
0.000 |     0 |      101 | 20190819-Kutna Hora-L9-in-MVI_0043 [0, 1554]
0.000 |     0 |      131 | 20190819-Kutna Hora-L4-in-MVI_0013 [0, 1542]
0.000 |     0 |       47 | 20190819-Ricany-L9-out-MVI_0011 [0, 1554]
0.000 |     0 |    

In [9]:
val_outputs[:, 1].astype(int).mean()

2.4

In [10]:
print_outputs(val_outputs)

 rvce | error | n_events | file
– – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – 
0.000 |     0 |        2 | 20190819-Kutna Hora-L15-out-MVI_0012 [1548, 2052]
0.000 |     0 |       15 | 20190819-Ricany-L9-in-MVI_0008 [1536, 2064]
0.000 |     0 |        1 | 20190819-Kutna Hora-L13-out-MVI_0018 [1680, 2046]
0.000 |     0 |        2 | 20190819-Kutna Hora-L13-in-MVI_0006 [1620, 2040]
0.013 |    -1 |       77 | 20190819-Kutna Hora-L3-in-MVI_0005 [1518, 2058]
0.014 |     1 |       72 | 20190819-Kutna Hora-L10-out-SDV_1888 [2820, 3750]
0.019 |     3 |      158 | 20190819-Ricany-L2-out-MVI_0005 [1500, 2064]
0.024 |    -1 |       42 | 20190819-Kutna Hora-L14-out-MVI_0005 [1554, 2064]
0.027 |     3 |      111 | 20190819-Ricany-L2-in-MVI_0006 [1548, 2064]
0.033 |     5 |      151 | 20190819-Ricany-L7-in-MVI_0008 [1560, 2064]
0.038 |     1 |       26 | 20190819-Kutna Hora-L9-in-MVI_0043 [1554, 2064]
0.051 |    -8 |      158 | 20190819-Ricany-L6-out-MVI_0011 [1566